In [1]:
# -- coding: utf-8 --
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import math
import json
import tqdm
import os
import random
import shutil
import time
import torch
import torchvision
import splitfolders
from torch import nn, optim
from PIL import Image
from torch.utils.data import Dataset

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
# data path
photo_path = '/Users/imchengliang/Downloads/DS-Project/photo/'
train_path = '/Users/imchengliang/Downloads/DS-Project/b_w_image/'
json_path = '/Users/imchengliang/Downloads/DS-Project/json-data/'
train_path = '/Users/imchengliang/Downloads/DS-Project/b_w_image/'
data_path = '/Users/imchengliang/Downloads/DS-Project/data/'
data_path_1 = '/Users/imchengliang/Downloads/DS-Project/train_0/'

## Save coordinate data into csv

In [6]:
def generate_train_df():
    file_list = sorted(os.listdir(json_path))
    info_list = []
    for f in file_list:
        path = json_path + '/' + f
        p = f.split(".")
        p_path = photo_path + p[0] + '.png'
        with open(path) as data:
            dict1 =  json.load(data)
            for i in range(len(dict1['shapes'])):
                info = {}
                info['number'] = p[0]
                info['jsonname'] = path
                info['photoname'] = p_path
                info['width'] = dict1['imageWidth']
                info['height'] = dict1['imageHeight']
                info['class'] = dict1['shapes'][i]['label']
                info['xmin'] = dict1['shapes'][i]['points'][0][0]
                info['ymin'] = dict1['shapes'][i]['points'][0][1]
                info['xmax'] = dict1['shapes'][i]['points'][1][0]
                info['ymax'] = dict1['shapes'][i]['points'][1][1]
                info_list.append(info)
    return pd.DataFrame(info_list)


In [7]:
df_train = generate_train_df()
df_train.to_csv('data.csv', index=False)

(1708, 10)


## Switch the image into black and white

In [3]:
def preprocess(path):
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # binary conversions
    ret, threshold = cv2.threshold(gray, 160, 255, cv2.THRESH_BINARY)
    #kernel = np.ones((2, 2), np.uint8)
    # morphological operations:
    #erosion = cv2.erode(threshold, kernel, iterations = 1)
    #dilation = cv2.dilate(threshold, kernel,iterations = 1)
    #opening = cv2.morphologyEx(threshold, cv2.MORPH_OPEN, kernel)
    #closing = cv2.morphologyEx(threshold, cv2.MORPH_CLOSE, kernel)
    # rescale the size
    #new_width  = 700
    #new_height = 1000
    #resized_image = cv2.resize(threshold, (new_width, new_height))
    #cv2.imshow("cvtColor2gray", closing)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    #plt.imshow(closing, 'gray')
    #plt.show()
    a = path.split('/')[-1]
    cv2.imwrite('/Users/imchengliang/Downloads/DS-Project/b_w_image/'+a, threshold, [cv2.IMWRITE_PNG_COMPRESSION, 0])

In [8]:
file_list = sorted(os.listdir(photo_path))
for i in file_list:
    preprocess('/Users/imchengliang/Downloads/DS-Project/photo/'+i)

## Split training and test set

In [17]:
train_dir = './train/train_image'
test_dir = './test/test_image'
image_dir = './b_w_image'

In [15]:
file_list = sorted(os.listdir(image_dir))
random.seed(42)
random.shuffle(file_list)
num_samples = len(file_list)
train_size = 0.9
train_data = file_list[0:math.ceil(train_size*num_samples)]
test_data = file_list[math.ceil(train_size*num_samples):]

for i in train_data:
    shutil.copy(image_dir+'/'+i, train_dir+'/'+i)
for i in test_data:
    shutil.copy(image_dir+'/'+i, test_dir+'/'+i)

## Save the coordination data into a dictionary

In [20]:
def save_coord(id, name):
    file = pd.read_csv('data.csv')
    #id = file['number'].unique()
    coordinates = {}
    for j in id:
        i = j.split('.')[0]
        a = file.loc[file['number']==int(i)]
        coordinate = []
        for index, row in a.iterrows():
            b = []
            b.append(row['xmin'])
            b.append(row['ymin'])
            b.append(row['xmax'])
            b.append(row['ymax'])
            #b.append(row['class'])
            coordinate.append(b)
        coordinates[int(i)] = coordinate

    with open(name, 'w') as f:
        json.dump(coordinates, f)

In [21]:
train_id = sorted(os.listdir(train_dir))
test_id = sorted(os.listdir(test_dir))
train_name = 'train_coordinates.json'
test_name = 'test_coordinates.json'
save_coord(train_id, train_name)
save_coord(test_id, test_name)

In [22]:
with open('train_coordinates.json') as f:
    train_coordinates = json.load(f)

with open('test_coordinates.json') as f:
    test_coordinates = json.load(f)

## Cut the handwritten part and save its shape information

In [30]:
def cut_handwritten(t_coordinates, file, path1, path2):
    cut_shape = {}
    for i in t_coordinates.keys():
        print(i)
        img_path = path1+'/'+str(i)+'.png'
        im = cv2.imread(img_path)
        a = 1
        for j in t_coordinates[i]:
            name = str(i)+'_'+str(a)
            # y first, x second
            ims = im[math.ceil(j[1]):math.ceil(j[3]), math.ceil(j[0]):math.ceil(j[2])]
            cut_shape[name] = [ims.shape[0], ims.shape[1]]
            cv2.imwrite(path2+str(i)+'_'+str(a)+'.png', ims)
            a += 1

    with open(file, 'w') as f:
        json.dump(cut_shape, f)

In [ ]:
train_cut_dir = './train/train_cut/'
test_cut_dir = './test/test_cut/'
cut_handwritten(train_coordinates, 'train_cut_shape.json', train_dir, train_cut_dir)
cut_handwritten(test_coordinates, 'test_cut_shape.json', test_dir, test_cut_dir)

In [32]:
with open('train_cut_shape.json') as f:
    train_cut_shape = json.load(f)

with open('test_cut_shape.json') as f:
    test_cut_shape = json.load(f)

## Cut and reshape the handwritten part

In [33]:
def pre_data(path1, path2, j):
    img = cv2.imread(path1+j+'.png')
    kernel = np.ones((2, 2), np.uint8)
    opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    if (opening.shape[0]>=opening.shape[1] and (opening.shape[0]/opening.shape[1])<=2.07) or (opening.shape[0]<=opening.shape[1] and (opening.shape[1]/opening.shape[0])<=2.07):
        resized_image = cv2.resize(opening, (227, 227))
        cv2.imwrite(path2+j+'_'+'.png', resized_image)
    elif opening.shape[0]>opening.shape[1] and (opening.shape[0]/opening.shape[1])>2.07:
        n = int(opening.shape[0] / opening.shape[1])
        dis_h = int(np.floor(opening.shape[0] / n))
        for i in range(n):
            a = i+1
            sub = opening[i*dis_h:(i+1)*dis_h, 0:opening.shape[1]]
            resized_image = cv2.resize(sub, (227, 227))
            cv2.imwrite(path2+j+'_'+str(a)+'.png', resized_image)
    elif opening.shape[0]<opening.shape[1] and (opening.shape[1]/opening.shape[0])>2.07:
        n = int(opening.shape[1] / opening.shape[0])
        dis_w = int(np.floor(opening.shape[1] / n))
        for i in range(n):
            a = i+1
            sub = opening[0:opening.shape[0], i*dis_w:(i+1)*dis_w]
            resized_image = cv2.resize(sub, (227, 227))
            cv2.imwrite(path2+j+'_'+str(a)+'.png', resized_image)
    print(j)


In [ ]:
train_1_dir = './train/train_1/'
test_1_dir = './test/test_1/'

for j in train_cut_shape.keys():
    pre_data(train_cut_dir, train_1_dir, j)

for j in test_cut_shape.keys():
    pre_data(test_cut_dir, test_1_dir, j)

## Check the maximum and minium of box size

In [3]:
file = pd.read_csv('data.csv')
x, y, xy = [], [], []
for i in range(len(file['number'])):
    x.append(file['xmax'][i]-file['xmin'][i])
    y.append(file['ymax'][i]-file['ymin'][i])
    xy.append((file['xmax'][i]-file['xmin'][i])*(file['ymax'][i]-file['ymin'][i]))
print(max(x), min(x), max(y), min(y), max(xy), min(xy))

3528.0 41.176470588235304 3703.703703703703 33.33333333333337 7492408.329560887 1614.7635524798125
